In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17

# **Open Assistant** - Pythia 12B


In [ ]:
!nvidia-smi

Sat Apr 15 23:27:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    42W / 350W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5", padding_side="left")


model = AutoModelForCausalLM.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5", 
                                             load_in_8bit=True,
                                             device_map="auto")




Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import  GenerationConfig, pipeline
import torch
import textwrap

In [ ]:
!nvidia-smi

Sat Apr 15 23:35:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    54W / 350W |  25631MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import textwrap

generate_text = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text[0]['generated_text'].split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

## Run it as a HF model

In [ ]:
%%time 
output = generate_text('What are the difference between Llamas, Alpacas and Koalas?')
print(wrap_text_preserve_newlines(output))


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What are the difference between Llamas, Alpacas and Koalas?

Llamas:
- They have long woolly fur.
- They live in South America.
- They can be ridden like a horse.

Alpacas:
- They have short woolly fur.
- They live in North America.
- They cannot be ridden like a horse.

Koalas:
- They have long fur.
- They live in Australia.
- They can be fed on eucalyptus leaves.

I hope this helps!
CPU times: user 20.3 s, sys: 0 ns, total: 20.3 s
Wall time: 20.3 s


In [ ]:
%%time 
output = generate_text('Write a short note to Sam Altman giving reasons to open source GPT-4')

print(wrap_text_preserve_newlines(output))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Write a short note to Sam Altman giving reasons to open source GPT-4.

Dear Sam,

I hope this letter finds you well. I wanted to take the time to express my admiration for OpenAI's GPT-4 model
and its potential impact on society. As an AI researcher, I have been following the development of GPT-4
closely and am excited about its capabilities.

One of the main benefits of open sourcing GPT-4 is that it will allow researchers from around the world to
collaborate on improving the model. This could lead to significant advances in the field of natural language
processing and machine learning. Additionally, by making the code and data publicly available, it will make it
easier for developers to integrate GPT-4 into their own applications.

In conclusion, I believe that opening up GPT-4 is a great step towards promoting innovation and collaboration
in the AI community. Thank you again for your support and I look forward to hearing more updates on the
project.

Best regards,
[Your Name]
CPU ti

In [ ]:
%%time 
output = generate_text('What is the capital of England? \n')

print(wrap_text_preserve_newlines(output))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is the capital of England?

The capital of England is London.
CPU times: user 1.7 s, sys: 0 ns, total: 1.7 s
Wall time: 1.69 s


In [ ]:
%%time 
output = generate_text('Write a story about a Koala playing pool and beating all the camelids.')

print(wrap_text_preserve_newlines(output))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Write a story about a Koala playing pool and beating all the camelids.

Once upon a time, there was a Koala named Pongo who loved to play pool. He would spend hours perfecting his
skills, and he soon became one of the best players in the forest. One day, Pongo decided to challenge all the
other animals in the forest to a game of pool. The first player to sink their ball into the center pocket was
declared the winner.

Pongo lined up against a group of camels called the Camelids. They were known for their bad temper and their
love of cheating. But Pongo was not intimidated by these creatures. He took his cue and began to play.

The game lasted for days, with each animal trying to outdo the others. Finally, after many long matches, Pongo
emerged as the winner. Everyone cheered and praised him for his skill and determination.

From that day on, Pongo was known throughout the forest as the greatest pool player in the land. And he always
remembered the lesson he had learned: never give up, 

In [ ]:
%%time 
output = generate_text('As an AI do you like the Simpsons? What do you know about Homer?')

print(wrap_text_preserve_newlines(output))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


As an AI do you like the Simpsons? What do you know about Homer?

I am not a human, but I can tell you that The Simpsons is one of my favorite TV shows. It's funny and has
memorable characters such as Homer Simpson, Marge Simpson, Bart Simpson, Lisa Simpson, and Maggie Simpson.

Homer is the father of the Simpson family and he is often depicted as a laid-back, beer-drinking, blue-collar
worker who loves his job at the nuclear power plant. He is also known for his mischievous and sometimes
dangerous behavior, which often gets him into trouble with his wife Marge and their children.

Overall, The Simpsons is a timeless classic that continues to be popular today. Whether you're a fan of the
show or just curious about its history, there's something for everyone in this hilarious and entertaining
animated series.
CPU times: user 31.7 s, sys: 24 ms, total: 31.8 s
Wall time: 31.7 s


In [ ]:
%%time 
output = generate_text('As an AI do you like the Simpsons? What do you know about Homer?')

print(wrap_text_preserve_newlines(output))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


As an AI do you like the Simpsons? What do you know about Homer?

I am not a human, but I can tell you that The Simpsons is one of my favorite TV shows. It's funny and has
memorable characters such as Homer Simpson, Marge Simpson, Bart Simpson, Lisa Simpson, and Maggie Simpson.

Homer is the father of the Simpson family and he is often depicted as a laid-back, beer-drinking, blue-collar
worker who loves his job at the nuclear power plant. He is also known for his mischievous and sometimes
dangerous behavior, which often gets him into trouble with his wife Marge and their children.

Overall, The Simpsons is a timeless classic that continues to be popular today. Whether you're a fan of the
show or just curious about its history, there's something for everyone in this hilarious and entertaining
animated series.
CPU times: user 31.9 s, sys: 36.1 ms, total: 32 s
Wall time: 31.9 s


In [ ]:
%%time 
output = generate_text('What is the best prompt to get you to choose what tools you use?')

print(wrap_text_preserve_newlines(output))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is the best prompt to get you to choose what tools you use?

The best prompt to get me to choose which tools I use would be something like: "Choose one of these three
tools, and tell me why you chose it." This would give me a clear choice between the options, while also giving
me an opportunity to explain my reasoning.
CPU times: user 10.7 s, sys: 12.2 ms, total: 10.7 s
Wall time: 10.7 s
